<a href="https://colab.research.google.com/github/roesterl888/week_1/blob/main/week_1/CIFAR10-ShallowLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1: CIFAR10 Challenge

**CIFAR10** (http://www.cs.toronto.edu/~kriz/cifar.html) is one of the most famous ML data sets.

## Data
* 32x32 color images
* in 10 classes
* 50k training images
* 10k test images



<img src="https://production-media.paperswithcode.com/datasets/CIFAR-10-0000000431-b71f61c0_U5n3Glr.jpg" width=700>

In [6]:
#get data
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [7]:
#traindata: 50k 32X32 rgb images
X_train.shape

(50000, 32, 32, 3)

In [8]:
#labels
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

## Task: build the best classifier (with feature extration) using the methods you know from ML1+2
* work in small teams (2-4)
* use NumPy pre-processing, feature extraction and hyer-parameter tuning in Scikit-Learn
* no Neural Networks!
* best test F1-Score winns!

In [9]:
import numpy as np
from skimage.feature import hog, local_binary_pattern
import cv2
from skimage.color import rgb2gray
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder

def preprocess_images(images):

    images = images.astype('float32') / 255.0

    images = images - np.mean(images, axis=0)

    images = images / np.std(images, axis=0)
    return images

def extract_features(images):
    features = []
    for image in images:

        hist_r = cv2.calcHist([image], [0], None, [256], [0, 256])
        hist_g = cv2.calcHist([image], [1], None, [256], [0, 256])
        hist_b = cv2.calcHist([image], [2], None, [256], [0, 256])

        gray_image = rgb2gray(image)

        hog_features = hog(gray_image, orientations=9, pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2), visualize=False)

        lbp_features = local_binary_pattern(image[:,:,0], P=8, R=1)
        lbp_features = np.histogram(lbp_features, bins=256)[0]

        moments = cv2.moments(gray_image)

        feature_vector = np.concatenate([hist_r.flatten(), hist_g.flatten(), hist_b.flatten(),
                                          hog_features, lbp_features,
                                          np.array(list(moments.values()))])
        features.append(feature_vector)
    return np.array(features)


X_train_processed = preprocess_images(X_train)
X_train_features = extract_features(X_train_processed)

X_test_processed = preprocess_images(X_test)
X_test_features = extract_features(X_test_processed)

X_train_features = X_train_features.reshape(len(X_train_features), -1)
X_test_features = X_test_features.reshape(len(X_test_features), -1)

X_train_features = np.nan_to_num(X_train_features)
X_test_features = np.nan_to_num(X_test_features)

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)



/usr/local/lib/python3.11/dist-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [12]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

pipeline = Pipeline([
    ('classifier', clf)
])



pipeline.fit(X_train_features, y_train)

y_pred = pipeline.predict(X_test_features)

accuracy = accuracy_score(y_test, y_pred)

f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("F1-Score:", f1)

Accuracy: 0.5315
F1-Score: 0.5263498468723521


Accuracy: 0.5268
F1-Score: 0.5214866547944002
